Про ноутбук
==
В цьому ноутбуці ми наводимо приклади найпростіших методів сортування та рангування.
Починаючи від самого простого підходу, ми переходимо до оцінки Байєса.

Імпортуємо необхідні бібліотеки
--

In [97]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.graph_objects import *
from scipy.stats import beta

Зчитуємо перший датасет
--

In [98]:
df = pd.read_csv('datasets/impressions_bookings.csv')

In [99]:
df.columns

Index(['Book ID', 'Impressions', 'Bookings', 'Year of Publishing', 'Price'], dtype='object')

In [100]:
df.head()

,Book ID,Impressions,Bookings,Year of Publishing,Price
0,1,312,47,2000,15.85
1,2,265,24,2004,17.85
2,3,324,38,2017,16.84
3,4,389,34,1999,26.64
4,5,257,72,1996,15.50


Колнки
---
* Book ID - ідентифікатор книги
* Impressions - кількість разів, коли книгу побачили
* Bookings - кількість разів, коли книгу придбали
* Year of Publishing - рік публікації
* Price - ціна

Статистика датасету
--

In [101]:
df.describe()

,Book ID,Impressions,Bookings,Year of Publishing,Price
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
mean,500.500000,273.967000,69.194000,2006.169000,20.20233
std,288.819436,77.108817,39.390862,9.553309,5.87326
min,1.000000,15.000000,1.000000,1990.000000,10.03000
25%,250.750000,224.000000,38.000000,1998.000000,15.26500
50%,500.500000,276.000000,63.000000,2006.000000,20.36000
75%,750.250000,323.000000,95.000000,2014.000000,25.30750
max,1000.000000,500.000000,188.000000,2022.000000,29.99000


In [102]:
df['cvr'] = df['Bookings']/df['Impressions']

In [103]:
df.sort_values(by='cvr', ascending=False, inplace=True)

In [104]:
df.head(10)

,Book ID,Impressions,Bookings,Year of Publishing,Price,cvr
466,467,16,9,1991,11.19,0.562500
890,891,41,23,1995,28.36,0.560976
541,542,36,20,1996,28.21,0.555556
40,41,27,15,2005,28.29,0.555556
198,199,29,16,1995,21.54,0.551724
80,81,20,11,1995,14.26,0.550000
505,506,33,18,2003,14.19,0.545455
332,333,24,13,2004,27.93,0.541667
586,587,24,13,2018,14.83,0.541667
181,182,15,8,2006,17.55,0.533333


Гістограма конверсій
--

In [105]:
px.histogram(df,
             x='cvr')

Зтворюємо дані, на яких ми будемо робити оцінку параметрів для бета-розподілу.
Нам треба відкинути ті рядки, де статистика невелика. 

In [106]:

data = df.query('Impressions > 0')['cvr']

Тепер ми шукаємо оцінку параметрів для розподілу бета. 

In [107]:
a, b, _, _ = beta.fit(data, floc=0, fscale=1)

In [108]:
a

2.8274176102633386

In [109]:
b

8.44193185162287

Математичне очікування бета функції

In [110]:
a/(a+b)

0.25089448329079495

Зтворимо вектор x, який має границі наших даних, та вектор y - значення функції ймовірності бета розподілу для всіх значень x. Ми це робимо лише для візуалізації. 

In [111]:
x = np.linspace(data.min(),
                data.max(),
                100)
y = beta.pdf(x,a,b)

In [112]:
trace1 = Histogram(x = data,
                   histnorm='probability density')
trace2 = Scatter(x = x,
                 y = y)
fig=Figure(data=[trace1, trace2])
fig.show()

In [113]:
df['updated_cvr'] = (df['Bookings']+a)/(df['Impressions']+a+b)
df.sort_values(by='updated_cvr', ascending=False, inplace=True)

In [114]:
df.head(100)

,Book ID,Impressions,Bookings,Year of Publishing,Price,cvr,updated_cvr
890,891,41,23,1995,28.36,0.560976,0.494122
886,887,319,158,2022,15.57,0.495298,0.486958
541,542,36,20,1996,28.21,0.555556,0.482922
286,287,264,127,1992,12.37,0.481061,0.471638
508,509,279,134,2004,24.31,0.480287,0.471381
...,...,...,...,...,...,...,...
958,959,318,131,1996,25.88,0.411950,0.406438
358,359,299,123,2019,29.44,0.411371,0.405543
124,125,319,131,1998,22.42,0.410658,0.405207
680,681,329,135,2011,19.07,0.410334,0.405054


In [25]:
df[df['Impressions']<50]

,Book ID,Impressions,Bookings,Year of Publishing,Price,cvr,updated_cvr
890,891,41,23,1995,28.36,0.560976,0.491361
541,542,36,20,1996,28.21,0.555556,0.479972
505,506,33,18,2003,14.19,0.545455,0.467440
198,199,29,16,1995,21.54,0.551724,0.464240
40,41,27,15,2005,28.29,0.555556,0.462392
332,333,24,13,2004,27.93,0.541667,0.445223
586,587,24,13,2018,14.83,0.541667,0.445223
80,81,20,11,1995,14.26,0.550000,0.438311
466,467,16,9,1991,11.19,0.562500,0.429401
181,182,15,8,2006,17.55,0.533333,0.408026


Завантажимо користувачів
----

In [26]:
users = pd.read_csv('datasets/users_books.csv')

In [85]:
users = users + users[['User ID','Book ID']] 


In [115]:
def recommend(user_id, top_n = 5):
    return df[~df['Book ID'].isin(users[users['User ID']==user_id]['Book ID'])]['Book ID'][0:top_n].values.tolist()

In [116]:
user_id = 50
print(f'For user {user_id} recommend {recommend(user_id)}')

For user 50 recommend [891, 887, 542, 287, 509]
